In [1]:
import sys
from dotenv import load_dotenv
from github import Github
import os

# Create a .env with the following content:
# GH_TOKEN=your_github_token
# REPON=$gh_org/$repo

# Load environment variables from .env file
load_dotenv("../.env")


# Get GitHub token from environment variable
gh_token = os.getenv("GH_TOKEN")
if gh_token is None:
    print("GitHub token not found! Create a .env file a the root with a GH_TOKEN variable.")
    exit(1)
gh_client = Github(gh_token, timeout=60)
repo = os.getenv("REPO") or "gh_org/repo"
repo = gh_client.get_repo(repo)

In [2]:
import sys
sys.path.append('..')
import importlib
import datagit.drift_evaluators
importlib.reload(datagit.drift_evaluators)

import datagit.github_connector
importlib.reload(datagit.github_connector)
from datagit.github_connector import store_metric
import pandas as pd

## Test with file already existing and splitting new data and historical data
repo = os.getenv("REPO") or "gh_org/repo"

def formatDF(dict):
    df = pd.DataFrame(dict)
    df['unique_key'] = df.apply(lambda row: row['date'] + '-' + row['name'], axis=1)
    column_order = ['unique_key'] + [col for col in df.columns if col != 'unique_key']
    df = df.reindex(columns=column_order)
    return df


file_path = repo+"/path/to/metric_name_13.csv"

# Store metric for the first time
dataMonth1 = {"name": ["Alice", "Bob", "Charlie"], "date": ["2022-12","2023-01","2023-01"], "age": [25, 30, 35]}
store_metric(ghClient=gh_client, dataframe= formatDF(dataMonth1),filepath= file_path, assignees=["Sammy"])




Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/metric_name_13.csv?token=ABUWFP6VI6D2TIZQRBV6FZ3FKOHSW
Dataframe dtypes {'name': string[python], 'date': string[python], 'age': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'name': string[python], 'date': string[python], 'age': string[python]}
Change detected
Update: DRIFT
https://github.com/Samox/data-history/commit/e033011d4294d823cceca58dda6cd704cdf29885


In [3]:
# ## Introduce new data for 2023-02
dataMonth2 = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02"], "age": [25, 30, 35, 40, 40, 40]}
store_metric(ghClient=gh_client,dataframe=  formatDF(dataMonth2),filepath= file_path, assignees=["Sammy"])



Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/metric_name_13.csv?token=ABUWFPZ2EE3DAEG4YCVZKITFKOHS2
Dataframe dtypes {'name': string[python], 'date': string[python], 'age': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'name': string[python], 'date': string[python], 'age': string[python]}
Change detected
Update: NEW DATA
https://github.com/Samox/data-history/commit/ed2b561758cd990ea793a09cc8110a454ee1d2aa


In [4]:
import sys
sys.path.append('..')
import importlib
import datagit.github_connector
importlib.reload(datagit.github_connector)
from datagit.github_connector import store_metric
import pandas as pd

# ## Introduce new data for 2023-03 and a drift on 2020-02
dataMonth3 = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 30, 36, 40, 42, 40, 45, 45, 46]}
store_metric(ghClient=gh_client,dataframe=  formatDF(dataMonth3),filepath= file_path, assignees=["Sammy"])



Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/metric_name_13.csv?token=ABUWFP7P2YPOOT6IKQIWQX3FKOHS6
Dataframe dtypes {'name': string[python], 'date': string[python], 'age': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'name': string[python], 'date': string[python], 'age': string[python]}
Change detected
Update: NEW DATA
https://github.com/Samox/data-history/commit/fa9f3a3c79835ac85b76459c9475829ec31bd52f
Update: DRIFT
https://github.com/Samox/data-history/commit/a30ba48f4b22761ee89382362c265c69058c8362


In [5]:

from datagit.dataframe_update_breakdown import dataframe_update_breakdown


dataframe_update_breakdown(initial_dataframe=formatDF(dataMonth2), final_dataframe=formatDF(dataMonth3))

{'MIGRATION Column Deleted': {'df':                     name     date  age
  unique_key                            
  2022-12-Alice      Alice  2022-12   25
  2023-01-Bob          Bob  2023-01   30
  2023-01-Charlie  Charlie  2023-01   35
  2023-02-Didier    Didier  2023-02   40
  2023-02-Philipe  Philipe  2023-02   40
  2023-02-Antoine  Antoine  2023-02   40,
  'has_update': False,
  'type': <UpdateType.OTHER: 'other'>,
  'drift_context': None,
  'drift_evaluation': None,
  'drift_summary': None},
 'NEW DATA': {'df':                     name     date  age
  unique_key                            
  2022-12-Alice      Alice  2022-12   25
  2023-01-Bob          Bob  2023-01   30
  2023-01-Charlie  Charlie  2023-01   35
  2023-02-Didier    Didier  2023-02   40
  2023-02-Philipe  Philipe  2023-02   40
  2023-02-Antoine  Antoine  2023-02   40
  2023-03-Clement  Clement  2023-03   45
  2023-03-Cyril      Cyril  2023-03   45
  2023-03-Victor    Victor  2023-03   46,
  'has_update': True,
  't

In [6]:
# ## No new data. Adds a drift for Philipe, and remove all other drifts
from datagit.drift_evaluators import alert_drift


dataMonth3And1Day = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 30, 35, 40, 42, 40, 45, 45, 46]}
store_metric(ghClient=gh_client, dataframe= formatDF(dataMonth3And1Day),filepath= file_path, assignees=["Sammy"], drift_evaluator=alert_drift)


Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/metric_name_13.csv?token=ABUWFP2WHXH7HDVRNBU72XDFKOHTG
Dataframe dtypes {'name': string[python], 'date': string[python], 'age': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'name': string[python], 'date': string[python], 'age': string[python]}
Change detected
Update: DRIFT
Branch drift/2023-11-14-16-15-02/path-to-metric-name-13-csv doesn't exist, creating it...
Checkout branch: drift/2023-11-14-16-15-02/path-to-metric-name-13-csv  from default branch:main
https://github.com/Samox/data-history/commit/244a9fda0b9bb735435fa1925ab14a4bc589bb8f
Pull request created: https://github.com/Samox/data-history/pull/176
Assignee Sammy does not exist


In [7]:
# ## No new data. Adds a drift for Philipe, and remove all other drifts
dataMonth3And2Day = {"name": ["Alice", "Alixe", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 25, 30, 35, 40, 42, 40, 45, 45, 46]}
store_metric(ghClient=gh_client, dataframe= formatDF(dataMonth3And2Day), filepath=file_path, assignees=["Sammy"], drift_evaluator=alert_drift)


Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/metric_name_13.csv?token=ABUWFPZ5QPZVUNQS2Z4UWRTFKOHTW
Dataframe dtypes {'name': string[python], 'date': string[python], 'age': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'name': string[python], 'date': string[python], 'age': string[python]}
Change detected
Update: DRIFT
Branch drift/2023-11-14-16-15-10/path-to-metric-name-13-csv doesn't exist, creating it...


In [ ]:
import pandas as pd
from faker import Faker
import random

# Initialize Faker to generate random data
fake = Faker()

# Set the number of rows for the dataframe
num_rows = 600000

# Generate random IDs and dates
ids = [fake.uuid4() for _ in range(num_rows)]
dates = [fake.date_between(start_date='-30y', end_date='today').strftime('%Y-%m-%d') for _ in range(num_rows)]

# Generate random metric values between 0 and 10
metric_values = [round(random.uniform(0, 10),2) for _ in range(num_rows)]

# Create the dataframe
ultra_large_df = pd.DataFrame({'unique_key': ids, 'date': dates, 'metric_value': metric_values})

# Print the dataframe
print(ultra_large_df)

                                  unique_key        date  metric_value
0       4a145858-84af-4451-a065-4e15455c6dd5  2008-07-16          3.79
1       842b637b-93b2-4341-a679-bf2c88105290  2014-01-20          7.92
2       e62492f5-ddfe-4237-a8b8-122cd4b49941  2000-10-23          5.86
3       eb43b6df-de49-47ef-a0c4-19ad05ab81e2  2010-11-06          7.64
4       0b390565-6c0f-46c4-822c-7cd1741a844f  1995-08-13          1.53
...                                      ...         ...           ...
599995  0d9f05b7-ddda-4f4f-bd4b-7e4b2f48f3e3  2019-11-07          5.68
599996  68a574c7-20d8-4e24-ad03-45b1e6b86e3a  2015-02-11          3.65
599997  c053adcb-badf-4569-9a20-7d5dca51681a  1996-08-02          0.18
599998  8d9ae907-6d61-41d1-90a5-461b1ba5dff8  2003-01-12          9.54
599999  876bca12-d852-4295-8b8e-afaa5be5c52c  2023-06-02          6.53

[600000 rows x 3 columns]


In [ ]:
import sys
sys.path.append('..')
import importlib
import datagit.github_connector
importlib.reload(datagit.github_connector)
from datagit.github_connector import store_metric
import pandas as pd

## Test with file already existing and splitting new data and historical data
repo = os.getenv("REPO") or "gh_org/repo"
store_metric(ghClient=gh_client,  dataframe=ultra_large_df, filepath=repo+"/path/to/ultra_large_metric_name3.csv", assignees=["Sammy"])

Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/ultra_large_metric_name3.csv?token=ABUWFP53PGSBFD5ZNKU45PTFKOHA2
Dataframe dtypes {'date': string[python], 'metric_value': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python]}
Change detected
Update: NEW DATA
https://github.com/Samox/data-history/commit/eaba558ad07f5f43d5931825816baa2a618c7c5f
Update: DRIFT
https://github.com/Samox/data-history/commit/78fe1770998890ebf8657bb3b6c6188423ac5027


In [ ]:
ultra_large_df = ultra_large_df.iloc[:-1]
